In [ ]:
import pandas as pd
import collections

def parse_usage(FILES, YEARS):

    RESULTS = {}
    YEARLY_USAGE= {}
    NEW_BUDGETS = {}
    ALL_BUDGETS = {}
    NEW_SRC_BUDGETS = {}
    ALL_SRC_BUDGETS = {}

    all_vre_budgets = set()
    all_budgets = set()

    for year in YEARS:

        df = pd.read_excel("{:s}/{:d}.xlsx".format(FILES, year), index_col=0, header=0)

        vre_and_snellius = df[df['Product'].str.startswith('research-cloud', na=False) | df['Product'].str.startswith('snellius', na=False)]
        budgets = set(vre_and_snellius.index.values)
        NEW_BUDGETS[year] = len(budgets - all_budgets)
        all_budgets.update(budgets)
        ALL_BUDGETS[year] = len(all_budgets)

        vre = df[df['Product'].str.startswith('research-cloud', na=False)]
        vre_budgets = set(vre.index.values)
        NEW_SRC_BUDGETS[year] = len(vre_budgets - all_vre_budgets)
        all_vre_budgets.update(vre_budgets)
        ALL_SRC_BUDGETS[year] = len(all_vre_budgets)
        
        cpu = vre[vre['SrvUnit'] == 'cpu-hr'].filter(regex=rf"{year}-\d+")
        gpu = vre[vre['SrvUnit'] == 'gpu-hr'].filter(regex=rf"{year}-\d+")

        gpu_conversion_rate = vre[vre['SrvUnit'] == 'gpu-hr']['Usage']/vre[vre['SrvUnit'] == 'gpu-hr']['SrvUsage']
        cpu_conversion_rate = vre[vre['SrvUnit'] == 'cpu-hr']['Usage']/vre[vre['SrvUnit'] == 'cpu-hr']['SrvUsage']
        print(year, " -> ", cpu_conversion_rate.mean(), " credits per CPU")
        print(year, " -> ", gpu_conversion_rate.mean(), " credits per GPU")

        for month in set(list(cpu.columns) + list(gpu.columns)):
            RESULTS[month] = {}
            RESULTS[month]['cpu_hours'] = cpu[month].sum()
            RESULTS[month]['gpu_hours'] = gpu[month].sum()
            RESULTS[month]['cpu_credits'] = (cpu[month]*cpu_conversion_rate.values).sum()
            RESULTS[month]['gpu_credits'] = (gpu[month]*gpu_conversion_rate.values).sum()
            RESULTS[month]['total'] = RESULTS[month]['gpu_credits'] + RESULTS[month]['cpu_credits']
            YEARLY_USAGE[year] = YEARLY_USAGE.get(year, 0) + RESULTS[month]['total']

    results = pd.DataFrame.from_dict(collections.OrderedDict(sorted(RESULTS.items())), orient='index')
    results.index = pd.to_datetime(results.index)
    yearly_usage = pd.DataFrame.from_dict(collections.OrderedDict(sorted(YEARLY_USAGE.items())), orient='index', columns=['total'])
    new_src_budgets = pd.DataFrame.from_dict(collections.OrderedDict(sorted(NEW_SRC_BUDGETS.items())), orient='index', columns=['new_src_budgets'])
    all_src_budgets = pd.DataFrame.from_dict(collections.OrderedDict(sorted(ALL_SRC_BUDGETS.items())), orient='index', columns=['all_src_budgets'])
    new_budgets = pd.DataFrame.from_dict(collections.OrderedDict(sorted(NEW_BUDGETS.items())), orient='index', columns=['new_budgets'])
    all_budgets = pd.DataFrame.from_dict(collections.OrderedDict(sorted(ALL_BUDGETS.items())), orient='index', columns=['all_budgets'])
    budgets = [new_src_budgets, all_src_budgets, new_budgets, all_budgets]
    #budgets = [df.set_index('year') for df in budgets]
    budgets = budgets[0].join(budgets[1:])

    return results, yearly_usage, budgets

def preprocess(df):
    """
    drop last row due to incomplete data, add month and year columns
    """
    df.drop(df.tail(1).index,inplace=True)
    df['month'] = df.index.month
    df['year'] = df.index.year
    return df

FILES = '../data'
YEARS = (2022, 2023, 2024)

results, yearly_usage, budgets = parse_usage(FILES, YEARS)
results = preprocess(results) 

print("YEARLY USAGE:")
print(yearly_usage)
print("MONTHLY USAGE:")
print(results)
print("NEW BUDGETS PER YEAR (VRE + SNELLIUS):")
print(budgets)


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker

fig, (ax1,ax2) = plt.subplots(nrows=1,ncols=2,figsize=(12,5))
ax1.plot(results['cpu_hours'], 'o-')
ax1.grid()
ax1.xaxis.set_tick_params(rotation=45, labelsize=10)
ax1.set_xlabel("Month", fontsize=14)
ax1.set_ylabel("Hours", fontsize=14)
ax1.set_title('CPU')
ax1.get_yaxis().set_major_formatter(
    ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

ax2.plot(results['gpu_hours'], 'o-', color='orange')
ax2.grid()
ax2.xaxis.set_tick_params(rotation=45, labelsize=10)
ax2.set_xlabel("Month", fontsize=14)
ax2.set_title('GPU')
ax2.get_yaxis().set_major_formatter(
    ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

fig.suptitle('Monthly Usage', fontsize=16)
fig.savefig('../presentations/images/monthly_usage.png', dpi=300, bbox_inches='tight')

In [ ]:


fig, ax = plt.subplots(figsize=(6, 6))

ax.plot(results[results['year']==2022]['month'],results[results['year']==2022]['total'], 'o-', label='2022')
ax.plot(results[results['year']==2023]['month'],results[results['year']==2023]['total'], 'o-', label='2023')
ax.plot(results[results['year']==2024]['month'],results[results['year']==2024]['total'], 'o-', label='2024')
ax.grid()
ax.legend()
ax.set_title('Credit usage', fontsize=16)
ax.set_xlabel('Month', fontsize=14)
ax.set_ylabel('Credits', fontsize=14)

In [ ]:
year = [2020, 2021, 2022, 2023]
totals = [200000, 280000, 500000, 900000]

fig, ax = plt.subplots(figsize=(6, 6))
ax.bar(year, totals, color='orange')
ax.set_xticks(year, labels=year)
ax.get_yaxis().set_major_formatter(
    ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_title('Credit usage', fontsize=16)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Credits', fontsize=14)

fig.savefig('../presentations/images/yearly_usage.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
budgets['all_src_budgets'].plot(kind='bar', color='orange', ax=ax)
ax.set_title('Research Cloud Projects', fontsize=16)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Budgets', fontsize=14)
ax.xaxis.set_tick_params(rotation=45, labelsize=10)
fig.savefig('../presentations/images/src_budgets.png', dpi=300, bbox_inches='tight')

In [ ]:
ax.bar(budgets, totals, color='orange')
ax.set_xticks(year, labels=year)
ax.get_yaxis().set_major_formatter(
    ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.set_title('Credit usage', fontsize=16)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Credits', fontsize=14)

fig.savefig('../presentations/images/yearly_usage.png', dpi=300)